In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

print("Imported all libraries")

Imported all libraries


# When downloading the datasets there was a issue where the download would fail after each gigabyte because i think they have some type of limit so i am going to load 2 of the same datasets and make sure that they both match

In [4]:
df1 = pd.read_csv('../../data/original_datasets/weather_data_original.csv')
# df2 = pd.read_csv('../../data/original_datasets/weather_data_original_backup.csv')

# print(f'No lost data from downloads: {df1.equals(df2)}')

In [5]:
# start analyzing the data to asses what needs to be done to have it prepped
df = df1
print(df.isnull().sum())
print('='*100)
print(f'shape: {df.shape}')
print('='*100)
print(df.dtypes)
print('='*100)
df.head(2)

dt                            0
dt_iso                        0
timezone                      0
city_name                     0
lat                           0
lon                           0
temp                          0
visibility             14861048
dew_point                     0
feels_like                    0
temp_min                      0
temp_max                      0
pressure                      0
sea_level              19241397
grnd_level             19241397
humidity                      0
wind_speed                    0
wind_deg                      0
wind_gust              18322567
rain_1h                17684504
rain_3h                19226218
snow_1h                19150527
snow_3h                19241302
clouds_all                    0
weather_id                    0
weather_main                  0
weather_description           0
weather_icon                  0
dtype: int64
shape: (19241397, 28)
dt                       int64
dt_iso                  object
timezon

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,283996800,1979-01-01 00:00:00 +0000 UTC,-28800,Alameda County,37.601689,-121.719546,6.63,NaN,-0.92,5.57,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01d
1,284000400,1979-01-01 01:00:00 +0000 UTC,-28800,Alameda County,37.601689,-121.719546,7.63,NaN,-3.67,7.63,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n


In [6]:
# based on the analysis of the data above i think that it is worth removing the columns with null values each of them are majorinly null
# so it could just end up skewing the data in incrorrect ways the one of the biggest relevance to not remove would be wind and rain
# but since they are so empty there would not be value in keepning them
# removing timezone and weather icon because they are redundant
# we also are goint to remove the dt col and later will change the dt_iso to a datetime object
col_to_drop = ['visibility', 'sea_level', 'grnd_level', 'wind_gust', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h', 'dt', 'weather_icon', 'timezone']
df = df.drop(col_to_drop, axis=1)
print(df.columns)
df.head()


Index(['dt_iso', 'city_name', 'lat', 'lon', 'temp', 'dew_point', 'feels_like',
       'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed',
       'wind_deg', 'clouds_all', 'weather_id', 'weather_main',
       'weather_description'],
      dtype='object')


,dt_iso,city_name,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description
0,1979-01-01 00:00:00 +0000 UTC,Alameda County,37.601689,-121.719546,6.63,-0.92,5.57,5.69,7.32,1024,58,1.70,14,0,800,Clear,sky is clear
1,1979-01-01 01:00:00 +0000 UTC,Alameda County,37.601689,-121.719546,7.63,-3.67,7.63,5.80,9.45,1023,43,1.19,59,0,800,Clear,sky is clear
2,1979-01-01 02:00:00 +0000 UTC,Alameda County,37.601689,-121.719546,6.93,-2.69,6.93,3.75,9.11,1024,49,0.43,111,0,800,Clear,sky is clear
3,1979-01-01 03:00:00 +0000 UTC,Alameda County,37.601689,-121.719546,-0.11,-4.71,-0.11,-1.63,1.41,1025,68,0.58,156,0,800,Clear,sky is clear
4,1979-01-01 04:00:00 +0000 UTC,Alameda County,37.601689,-121.719546,1.05,-4.79,1.05,-0.88,2.99,1025,62,1.01,132,0,800,Clear,sky is clear


In [7]:
#change the dt_iso to a datetime object then break the datetime down into year month day hour columns for easier future parsing

df['dt_iso'] = df['dt_iso'].str.replace(' UTC', '')
df['dt_iso'] = pd.to_datetime(df['dt_iso'], format='%Y-%m-%d %H:%M:%S %z', utc=True)
df['Year'] = df['dt_iso'].dt.year
df['Month'] = df['dt_iso'].dt.month
df['Day'] = df['dt_iso'].dt.day
df['Hour'] = df['dt_iso'].dt.hour
df = df.drop('dt_iso', axis=1)

In [8]:
# currently these are all object and they are worth keeping so i will caterogize them numerically so that they can be encoded in training
# weather_main	weather_description	weather_icon
# doing so with one hot encoding

df1 = df.copy()
encoded = pd.get_dummies(df[['weather_main',	'weather_description', ]])
df = pd.concat([df, encoded], axis=1)
print(df.shape)
print(df.dtypes)
df.head()


(19241397, 75)
city_name                                               object
lat                                                    float64
lon                                                    float64
temp                                                   float64
dew_point                                              float64
                                                        ...   
weather_description_thunderstorm with light drizzle       bool
weather_description_thunderstorm with light rain          bool
weather_description_thunderstorm with rain                bool
weather_description_tornado                               bool
weather_description_very heavy rain                       bool
Length: 75, dtype: object


,city_name,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,...,weather_description_snow,weather_description_squalls,weather_description_thunderstorm,weather_description_thunderstorm with drizzle,weather_description_thunderstorm with heavy rain,weather_description_thunderstorm with light drizzle,weather_description_thunderstorm with light rain,weather_description_thunderstorm with rain,weather_description_tornado,weather_description_very heavy rain
0,Alameda County,37.601689,-121.719546,6.63,-0.92,5.57,5.69,7.32,1024,58,...,False,False,False,False,False,False,False,False,False,False
1,Alameda County,37.601689,-121.719546,7.63,-3.67,7.63,5.80,9.45,1023,43,...,False,False,False,False,False,False,False,False,False,False
2,Alameda County,37.601689,-121.719546,6.93,-2.69,6.93,3.75,9.11,1024,49,...,False,False,False,False,False,False,False,False,False,False
3,Alameda County,37.601689,-121.719546,-0.11,-4.71,-0.11,-1.63,1.41,1025,68,...,False,False,False,False,False,False,False,False,False,False
4,Alameda County,37.601689,-121.719546,1.05,-4.79,1.05,-0.88,2.99,1025,62,...,False,False,False,False,False,False,False,False,False,False


In [9]:
df.columns

Index(['city_name', 'lat', 'lon', 'temp', 'dew_point', 'feels_like',
       'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed',
       'wind_deg', 'clouds_all', 'weather_id', 'weather_main',
       'weather_description', 'Year', 'Month', 'Day', 'Hour',
       'weather_main_Clear', 'weather_main_Clouds', 'weather_main_Drizzle',
       'weather_main_Dust', 'weather_main_Fog', 'weather_main_Haze',
       'weather_main_Mist', 'weather_main_Rain', 'weather_main_Smoke',
       'weather_main_Snow', 'weather_main_Squall', 'weather_main_Thunderstorm',
       'weather_main_Tornado', 'weather_description_broken clouds',
       'weather_description_drizzle', 'weather_description_dust',
       'weather_description_extreme rain', 'weather_description_few clouds',
       'weather_description_fog', 'weather_description_freezing rain',
       'weather_description_haze',
       'weather_description_heavy intensity drizzle',
       'weather_description_heavy intensity rain',
       'weather_des

In [10]:
df = df.rename(columns={
    'city_name': 'County',
    'lat': 'Lat',
    'lon': 'Lon',
    'temp': 'Temp',
    'dew_point': 'Dewpt',
    'feels_like': 'Feels',
    'temp_min': 'Tmin',
    'temp_max': 'Tmax',
    'pressure': 'Pres',
    'humidity': 'Humid',
    'wind_speed': 'Windsp',
    'wind_deg': 'Windir',
    'clouds_all': 'Clouds',
    'weather_id': 'Wthrid',
    'weather_main': 'Wthrmain',
    'weather_description': 'Wthrdesc',
    'Year': 'Year',
    'Month': 'Month',
    'Day': 'Day',
    'Hour': 'Hour',
    'weather_main_Clear': 'Clr',
    'weather_main_Clouds': 'Clds',
    'weather_main_Drizzle': 'Drzl',
    'weather_main_Dust': 'Dust',
    'weather_main_Fog': 'Fog',
    'weather_main_Haze': 'Haze',
    'weather_main_Mist': 'Mist',
    'weather_main_Rain': 'Rain',
    'weather_main_Smoke': 'Smoke',
    'weather_main_Snow': 'Snow',
    'weather_main_Squall': 'Squal',
    'weather_main_Thunderstorm': 'Thndr',
    'weather_main_Tornado': 'Torn',
    'weather_description_broken clouds': 'Bclds',
    'weather_description_drizzle': 'Drzl',
    'weather_description_dust': 'Dust',
    'weather_description_extreme rain': 'Xrrain',
    'weather_description_few clouds': 'Fclds',
    'weather_description_fog': 'Fog',
    'weather_description_freezing rain': 'Frzrn',
    'weather_description_haze': 'Haze',
    'weather_description_heavy intensity drizzle': 'Hidrzl',
    'weather_description_heavy intensity rain': 'Hirain',
    'weather_description_heavy intensity shower rain': 'Hisr',
    'weather_description_heavy snow': 'Hisnow',
    'weather_description_light intensity drizzle': 'Lidrzl',
    'weather_description_light intensity shower rain': 'Lisr',
    'weather_description_light rain': 'Lrain',
    'weather_description_light shower snow': 'Lssnow',
    'weather_description_light snow': 'Lsnow',
    'weather_description_mist': 'Mist',
    'weather_description_moderate rain': 'Mrrain',
    'weather_description_overcast clouds': 'Oclds',
    'weather_description_proximity Sand/Dust Whirls': 'Psdwl',
    'weather_description_proximity moderate rain': 'Pmr',
    'weather_description_proximity shower rain': 'Psr',
    'weather_description_proximity squalls': 'Psql',
    'weather_description_proximity thunderstorm': 'Pthndr',
    'weather_description_proximity thunderstorm with rain': 'Ptr',
    'weather_description_proximity tornado': 'Ptorn',
    'weather_description_scattered clouds': 'Sclds',
    'weather_description_shower rain': 'Sr',
    'weather_description_shower snow': 'Ssnow',
    'weather_description_sky is clear': 'Sclr',
    'weather_description_smoke': 'Smoke',
    'weather_description_snow': 'Snow',
    'weather_description_squalls': 'Sqls',
    'weather_description_thunderstorm': 'Thndr',
    'weather_description_thunderstorm with drizzle': 'Thdrzl',
    'weather_description_thunderstorm with heavy rain': 'Thhr',
    'weather_description_thunderstorm with light drizzle': 'Thldrz',
    'weather_description_thunderstorm with light rain': 'Thlr',
    'weather_description_thunderstorm with rain': 'Thr',
    'weather_description_tornado': 'Torn',
    'weather_description_very heavy rain': 'Vhr'
})
df.head()

,County,Lat,Lon,Temp,Dewpt,Feels,Tmin,Tmax,Pres,Humid,...,Snow,Sqls,Thndr,Thdrzl,Thhr,Thldrz,Thlr,Thr,Torn,Vhr
0,Alameda County,37.601689,-121.719546,6.63,-0.92,5.57,5.69,7.32,1024,58,...,False,False,False,False,False,False,False,False,False,False
1,Alameda County,37.601689,-121.719546,7.63,-3.67,7.63,5.80,9.45,1023,43,...,False,False,False,False,False,False,False,False,False,False
2,Alameda County,37.601689,-121.719546,6.93,-2.69,6.93,3.75,9.11,1024,49,...,False,False,False,False,False,False,False,False,False,False
3,Alameda County,37.601689,-121.719546,-0.11,-4.71,-0.11,-1.63,1.41,1025,68,...,False,False,False,False,False,False,False,False,False,False
4,Alameda County,37.601689,-121.719546,1.05,-4.79,1.05,-0.88,2.99,1025,62,...,False,False,False,False,False,False,False,False,False,False


In [11]:
print(df['County'].unique())

['Alameda County' 'Amador County' 'Berkeley' 'Butte Valley'
 'Calaveras County' 'Colusa County' 'Contra Costa County'
 'El Dorado County' 'Fresno County' 'Humboldt County' 'Imperial County'
 'Kern County' 'Lassen County' 'Long Beach' 'Madera County' 'Marin County'
 'Mendocino County' 'Merced County' 'Monterey County' 'Napa County'
 'Nevada County' 'Orange County' 'Pasadena' 'Placer County'
 'Riverside County' 'Sacramento County' 'San Benito County'
 'San Bernardino County' 'San Diego County' 'San Francisco County'
 'San Joaquin County' 'San Luis Obispo County' 'San Mateo County'
 'Santa Barbara County' 'Santa Clara County' 'Santa Cruz County'
 'Siskiyou County' 'Solano County' 'Sonoma County' 'Stanislaus County'
 'Sutter County' 'Tehama County' 'Tulare County' 'Tuolumne County'
 'Ventura County' 'Yolo County' 'Yuba County' 'Kings County']


In [12]:
#relabel all of the counties so it is in a better format

df['County'] = df['County'].replace({
    'Alameda County': 'Alameda',
    'Amador County': 'Amador',
    'Berkeley': 'Berkeley',
    'Butte Valley': 'Butte',
    'Calaveras County': 'Calaveras',
    'Colusa County': 'Colusa',
    'Contra Costa County': 'ContraCosta',
    'El Dorado County': 'ElDorado',
    'Fresno County': 'Fresno',
    'Humboldt County': 'Humboldt',
    'Imperial County': 'Imperial',
    'Kern County': 'Kern',
    'Lassen County': 'Lassen',
    'Long Beach': 'LongBeach',
    'Madera County': 'Madera',
    'Marin County': 'Marin',
    'Mendocino County': 'Mendocino',
    'Merced County': 'Merced',
    'Monterey County': 'Monterey',
    'Napa County': 'Napa',
    'Nevada County': 'Nevada',
    'Orange County': 'Orange',
    'Pasadena': 'Pasadena',
    'Placer County': 'Placer',
    'Riverside County': 'Riverside',
    'Sacramento County': 'Sacramento',
    'San Benito County': 'SanBenito',
    'San Bernardino County': 'SanBernardino',
    'San Diego County': 'SanDiego',
    'San Francisco County': 'SanFrancisco',
    'San Joaquin County': 'SanJoaquin',
    'San Luis Obispo County': 'SanLuisObispo',
    'San Mateo County': 'SanMateo',
    'Santa Barbara County': 'SantaBarbara',
    'Santa Clara County': 'SantaClara',
    'Santa Cruz County': 'SantaCruz',
    'Siskiyou County': 'Siskiyou',
    'Solano County': 'Solano',
    'Sonoma County': 'Sonoma',
    'Stanislaus County': 'Stanislaus',
    'Sutter County': 'Sutter',
    'Tehama County': 'Tehama',
    'Tulare County': 'Tulare',
    'Tuolumne County': 'Tuolumne',
    'Ventura County': 'Ventura',
    'Yolo County': 'Yolo',
    'Yuba County': 'Yuba',
    'Kings County': 'Kings'
})


In [13]:
print(df['County'].unique())

['Alameda' 'Amador' 'Berkeley' 'Butte' 'Calaveras' 'Colusa' 'ContraCosta'
 'ElDorado' 'Fresno' 'Humboldt' 'Imperial' 'Kern' 'Lassen' 'LongBeach'
 'Madera' 'Marin' 'Mendocino' 'Merced' 'Monterey' 'Napa' 'Nevada' 'Orange'
 'Pasadena' 'Placer' 'Riverside' 'Sacramento' 'SanBenito' 'SanBernardino'
 'SanDiego' 'SanFrancisco' 'SanJoaquin' 'SanLuisObispo' 'SanMateo'
 'SantaBarbara' 'SantaClara' 'SantaCruz' 'Siskiyou' 'Solano' 'Sonoma'
 'Stanislaus' 'Sutter' 'Tehama' 'Tulare' 'Tuolumne' 'Ventura' 'Yolo'
 'Yuba' 'Kings']


In [14]:
#convert bool col to floats
df = df.astype({col: 'float' for col in df.dtypes[df.dtypes == 'bool'].index})


In [15]:
print(df.dtypes)

County     object
Lat       float64
Lon       float64
Temp      float64
Dewpt     float64
           ...   
Thldrz    float64
Thlr      float64
Thr       float64
Torn      float64
Vhr       float64
Length: 75, dtype: object


In [16]:
df.head()

,County,Lat,Lon,Temp,Dewpt,Feels,Tmin,Tmax,Pres,Humid,...,Snow,Sqls,Thndr,Thdrzl,Thhr,Thldrz,Thlr,Thr,Torn,Vhr
0,Alameda,37.601689,-121.719546,6.63,-0.92,5.57,5.69,7.32,1024,58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alameda,37.601689,-121.719546,7.63,-3.67,7.63,5.80,9.45,1023,43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alameda,37.601689,-121.719546,6.93,-2.69,6.93,3.75,9.11,1024,49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alameda,37.601689,-121.719546,-0.11,-4.71,-0.11,-1.63,1.41,1025,68,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alameda,37.601689,-121.719546,1.05,-4.79,1.05,-0.88,2.99,1025,62,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df.to_csv('../../data/weather_datasets/one_hot_weather_06_02_24.csv', index=False)

In [18]:
# df1 is a copy of df before the onehot this one will be used for visualizations for its lower dimensionality
df = df1
df.head()

,city_name,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description,Year,Month,Day,Hour
0,Alameda County,37.601689,-121.719546,6.63,-0.92,5.57,5.69,7.32,1024,58,1.70,14,0,800,Clear,sky is clear,1979,1,1,0
1,Alameda County,37.601689,-121.719546,7.63,-3.67,7.63,5.80,9.45,1023,43,1.19,59,0,800,Clear,sky is clear,1979,1,1,1
2,Alameda County,37.601689,-121.719546,6.93,-2.69,6.93,3.75,9.11,1024,49,0.43,111,0,800,Clear,sky is clear,1979,1,1,2
3,Alameda County,37.601689,-121.719546,-0.11,-4.71,-0.11,-1.63,1.41,1025,68,0.58,156,0,800,Clear,sky is clear,1979,1,1,3
4,Alameda County,37.601689,-121.719546,1.05,-4.79,1.05,-0.88,2.99,1025,62,1.01,132,0,800,Clear,sky is clear,1979,1,1,4


In [19]:
df = df.rename(columns={
    'city_name': 'County',
    'lat': 'Lat',
    'lon': 'Lon',
    'temp': 'Temp',
    'dew_point': 'Dewpt',
    'feels_like': 'Feels',
    'temp_min': 'Tmin',
    'temp_max': 'Tmax',
    'pressure': 'Pres',
    'humidity': 'Humid',
    'wind_speed': 'Windsp',
    'wind_deg': 'Windir',
    'clouds_all': 'Clouds',
    'weather_id': 'Wthrid',
    'weather_main': 'Wthrmain',
    'weather_description': 'Wthrdesc',
    'Year': 'Year',
    'Month': 'Month',
    'Day': 'Day',
    'Hour': 'Hour'
})

df.head()

,County,Lat,Lon,Temp,Dewpt,Feels,Tmin,Tmax,Pres,Humid,Windsp,Windir,Clouds,Wthrid,Wthrmain,Wthrdesc,Year,Month,Day,Hour
0,Alameda County,37.601689,-121.719546,6.63,-0.92,5.57,5.69,7.32,1024,58,1.70,14,0,800,Clear,sky is clear,1979,1,1,0
1,Alameda County,37.601689,-121.719546,7.63,-3.67,7.63,5.80,9.45,1023,43,1.19,59,0,800,Clear,sky is clear,1979,1,1,1
2,Alameda County,37.601689,-121.719546,6.93,-2.69,6.93,3.75,9.11,1024,49,0.43,111,0,800,Clear,sky is clear,1979,1,1,2
3,Alameda County,37.601689,-121.719546,-0.11,-4.71,-0.11,-1.63,1.41,1025,68,0.58,156,0,800,Clear,sky is clear,1979,1,1,3
4,Alameda County,37.601689,-121.719546,1.05,-4.79,1.05,-0.88,2.99,1025,62,1.01,132,0,800,Clear,sky is clear,1979,1,1,4


In [20]:
# encode the strings as numeric values instead
encoder = LabelEncoder()

df['Wthrmain'] = encoder.fit_transform(df['Wthrmain'])
df['Wthrdesc'] = encoder.fit_transform(df['Wthrdesc'])
df['County'] = encoder.fit_transform(df['County'])
df.head()

,County,Lat,Lon,Temp,Dewpt,Feels,Tmin,Tmax,Pres,Humid,Windsp,Windir,Clouds,Wthrid,Wthrmain,Wthrdesc,Year,Month,Day,Hour
0,0,37.601689,-121.719546,6.63,-0.92,5.57,5.69,7.32,1024,58,1.70,14,0,800,0,30,1979,1,1,0
1,0,37.601689,-121.719546,7.63,-3.67,7.63,5.80,9.45,1023,43,1.19,59,0,800,0,30,1979,1,1,1
2,0,37.601689,-121.719546,6.93,-2.69,6.93,3.75,9.11,1024,49,0.43,111,0,800,0,30,1979,1,1,2
3,0,37.601689,-121.719546,-0.11,-4.71,-0.11,-1.63,1.41,1025,68,0.58,156,0,800,0,30,1979,1,1,3
4,0,37.601689,-121.719546,1.05,-4.79,1.05,-0.88,2.99,1025,62,1.01,132,0,800,0,30,1979,1,1,4


In [21]:
df.to_csv('../../data/weather_datasets/label_encoded_weather_06_02_24.csv', index=False)